### Importando Bibliotecas, tabela de busca e criando navegador

In [2]:
#imports, tabelas, e E-mail
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import win32com.client as win32
from pathlib import Path

#Criar um navegador
nav = webdriver.Chrome()

#Importar base de dados
tabela_produtos = pd.read_excel('buscas.xlsx')
display(tabela_produtos)

#Criando conexão com o Outlook
outlook = win32.Dispatch('outlook.application')

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,moto g60,g60s usado,1500,1800
1,motorola one hyper,usado,1500,1800
2,controle xbox one,360 series usado,200,270


### Criando a função da busca para o Google Shopping

In [3]:
def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    #Entrar no google
    nav.get('https://www.google.com/')
    
    #Tratar os valores que vieram da tabela
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_produto = produto.split(' ')
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    
    #Pesquisar o nome do produto no Google
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

    #Clicar na aba Shopping
    cabecalho_google = nav.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    for item in cabecalho_google:
        if 'Shopping' in item.text:
            item.click()
            break
    
    #pegando o elemento de todos os resultados
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'sh-dgr__gr-auto')
    
    #Criar lista para retorno
    lista_ofertas = []
    for resultado in lista_resultados:
        #Tratando o nome
        nome = resultado.find_element(By.CLASS_NAME, 'Xjkr3b').text
        nome = nome.lower()

        #Verificação do nome
        tem_termos_banidos = False
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos = True

        tem_todos_termos_produto = True
        for palavra in lista_termos_produto:
            if palavra not in nome:
                tem_todos_termos_produto = False
        
        try:
            if not tem_termos_banidos and tem_todos_termos_produto:
                #Tratando o preco
                preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
                preco =preco.replace('R$', "").replace(" ", '').replace('.', '').replace(',', '.')
                preco = float(preco)

                if preco_minimo <= preco <= preco_maximo:
                    elemento_link = resultado.find_element(By.CLASS_NAME, 'aULzUe')
                    elemento_pai = elemento_link.find_element(By.XPATH, '..')
                    link = elemento_pai.get_attribute('href')
                    lista_ofertas.append((nome, preco, link))
        except:
            continue
    return lista_ofertas

### Criando a função da busca para o Mercado Livre

In [4]:
def busca_mercado_livre(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    #entrar no site do mercado livre
    nav.get('https://www.mercadolivre.com.br/')

    #definir e tratar produto
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_produto = produto.split(' ')
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)

    #buscar o nome do produto
    nav.find_element(By.CLASS_NAME, 'nav-search-input').send_keys(produto)
    nav.find_element(By.CLASS_NAME, 'nav-search-input').send_keys(Keys.ENTER)

    #pegar elemento de todos os produtos
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'ui-search-layout__item')

    #Criar Lista de ofertas
    lista_ofertas = []
    try:
        for resultado in lista_resultados:
            #tratar nome do produto
            nome = resultado.find_element(By.CLASS_NAME, 'ui-search-item__title').text
            nome = nome.lower()

            #Verificar o nome
            tem_termos_banidos = False
            for palavra in lista_termos_banidos:
                if palavra in nome:
                    tem_termos_banidos = True

            tem_todos_termos_produto = True
            for palavra in lista_termos_produto:
                if palavra not in nome:
                    tem_todos_termos_produto = False

            if not tem_termos_banidos and tem_todos_termos_produto:
                try:
                    elemento_preco = resultado.find_element(By.CLASS_NAME, 'ui-search-price__second-line')
                    preco = elemento_preco.find_element(By.CLASS_NAME, 'price-tag-fraction').text
                    preco = preco.replace(".", "")
                    preco = float(preco)
                except:
                    pass

                if preco_minimo <= preco <= preco_maximo:
                    link = resultado.find_element(By.CLASS_NAME, 'ui-search-link').get_attribute('href')
                    lista_ofertas.append((nome, preco, link))
    except:
        pass
    return(lista_ofertas)

### Continuando o Código

In [5]:
tabela_ofertas = pd.DataFrame()
#Colocar para criar uma variácel para cada produto na tabela
for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, 'Nome']
    termos_banidos = tabela_produtos.loc[linha, 'Termos banidos']
    preco_minimo = tabela_produtos.loc[linha, 'Preço mínimo']
    preco_maximo = tabela_produtos.loc[linha, 'Preço máximo']
    
    #Colocar numa lista o return da função de busca do google
    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    #Colocar numa tabela SE a lista não estiver vazia
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=[r'Produtos', r'Preço', r'Link'])
        #Adicionar a tabela Ofertas
        tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
    else:
        tabela_google_shopping = None
    
    #Colocar numa lista o return da função de busca do Mercado Livre
    lista_ofertas_mercado_livre = busca_mercado_livre(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    #Colocar numa tabela SE a lista não estiver vazia
    if lista_ofertas_mercado_livre:
        tabela_mercado_livre = pd.DataFrame(lista_ofertas_mercado_livre, columns=[r'Produtos', r'Preço', r'Link'])
        #Adicionar a tabela Ofertas
        tabela_ofertas = tabela_ofertas.append(tabela_mercado_livre)
    else:
        tabela_mercado_livre = None
nav.close()

### Salvando em uma planilha, e enviando E-mail

In [6]:
#Salvar a tabela de ofertas
tabela_ofertas = tabela_ofertas.reset_index(drop=True)
tabela_ofertas.to_excel('Ofertas.xlsx', index=False)


#Verificar se tem algo na tabela de pfertas para enviar o E-mail
if len(tabela_ofertas.index) > 0:
    mail = outlook.CreateItem(0)
    mail.To = 'carlosdudu369@gmail.com'
    mail.Subject = 'Produtos Pesquisados'
    mail.HTMLBody = f"""
    <p>Boa tarde!</p>

    <p>Segue a baixo a tabela com os produtos da planilha:</p>

    {tabela_ofertas.to_html(index=False)}

    <p>Estou enviando também o arquivo em Excel caso queira visualizar melhor.</p>

    <p>Att..,</p>
    <pCarlos, Python Developer Jr.</p>.
    """

    # Anexos (pode colocar quantos quiser):
    attachment  = Path.cwd() / r'Ofertas.xlsx'
    mail.Attachments.Add(str(attachment))

    mail.Send()